In [3]:
import os

work_dir = '~/insarlab/PYSAR_EXAMPLE/AlcedoSenDT128/PYSAR'
work_dir = os.path.expanduser(work_dir)
os.chdir(work_dir)
print('Go to directory: '+work_dir)

plotCmd = 'view.py avgPhaseVelocity.h5 -m maskSpatialCoh.h5 --fontsize 12 --figsize 4 3 --notitle --dpi 600 --nodisplay'
print(plotCmd)
os.system(plotCmd)


Go to directory: /Users/yunjunz/insarlab/PYSAR_EXAMPLE/AlcedoSenDT128/PYSAR
view.py avgPhaseVelocity.h5 -m maskSpatialCoh.h5 --fontsize 12 --figsize 4 3 --notitle --dpi 600 --nodisplay


0